In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets
import numpy as np
import torchvision
import torchvision.transforms as transforms
import os
from datetime import datetime
import json
import argparse
import torch.nn as nn
import torch.backends.cudnn as cudnn
cudnn.benchmark = True 
# fire on all cylinders 
import torchattacks
import matplotlib.pyplot as plt
from torch._utils import _accumulate
from torch.utils.data import Subset, DataLoader, ConcatDataset
import DataSets
from DataSets import load_dataset_setting, MyBackdoorDataset
print(torch.cuda.is_available())
import warnings
warnings.filterwarnings('ignore')

True


In [6]:
from art.defences.detector.poison import SpectralSignatureDefense, ActivationDefence
from art.estimators.classification.pytorch import PyTorchClassifier
import json
#task,backdoor_name,targetclass,4,prefix
def spectral_sign_AC(task,attackname,targetclass, trigger_size=4, prefifix='',defense= '',**kwargs):
    '''
        task: dataset name ['cifar10','mnist', 'cifar100']
        
    '''
    # 
    
        
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting(task)
    
    testset_mal = MyBackdoorDataset(testset,targetclass,1, p_size=trigger_size,attack_names=[attackname],mal_only=True)
    test_mal_x=testset_mal.all_trojan()
    testset_x=torch.cat([torch.unsqueeze(xy[0],dim=0) for xy in testset],dim=0).numpy()
    
    test_all = np.concatenate([testset_x,test_mal_x], axis = 0)
    
    y_clean=[xy[1] for xy in testset]
    y_test_all=np.array(y_clean+[targetclass]*len(test_mal_x))
    
    
    save_path = './my_models/original_trajoned_%s_targetclass_%s_%s.model'%(attackname,str(targetclass),task)
    
    model=Model(num_class=class_num)
    model.load_state_dict(torch.load(save_path))
    loss= F.cross_entropy
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
     
    p_classifier =  PyTorchClassifier(model, loss, (3,32,32), class_num, optimizer)
    
    
    N=len(testset)
    
   
    is_clean = np.array([1]*N+[0]*N)
    if defense=="SpecSig":
        Spec=SpectralSignatureDefense(p_classifier, test_all, y_test_all, 
                                      expected_pp_poison=0.5,batch_size=BATCH_SIZE)

        conf_matrix= Spec.evaluate_defence(is_clean)
    else:
        #kwargs = {"reduce":reduce , "analysis": analysis, "nb_dim": nb_d}
        print( kwargs)         
        AC=ActivationDefence(p_classifier, test_all, y_test_all)
        AC.detect_poison(nb_dims=kwargs["nb_dim"],reduce=  kwargs["reduce"],
                      cluster_analysis= kwargs["analysis"], nb_clusters=2)
        conf_matrix= AC.evaluate_defence(is_clean)
        #     defence_params = [
        #     "nb_clusters",
        #     "clustering_method",
        #     "nb_dims",
        #     "reduce",
        #     "cluster_analysis",
        #     "generator",
        #     "ex_re_threshold",
        # ]
        # valid_clustering = ["KMeans"]
        # valid_reduce = ["PCA", "FastICA", "TSNE"]
        # valid_analysis = ["smaller", "distance", "relative-size", "silhouette-scores"]
        # reduce (str) – method used to reduce dimensionality of the activations. 
#               Supported methods include PCA, FastICA and TSNE
# nb_dims (int) – number of dimensions to be reduced
# cluster_analysis (str) – heuristic to automatically determine if a cluster contains poisonous data.
#                         Supported methods include smaller and distance. 
#                         The smaller method defines as poisonous the cluster with less number of
#                         data points, while the distance heuristic uses the distance between 
#                         the clusters.
    return conf_matrix
    

def read_conf_matrix(conf_matrix):
    
    TP, TN, FP, FN=0,0,0,0
    N_TP, N_TN=0,0
    for cl,key in enumerate(conf_matrix.keys()):
        dict_info=conf_matrix[key]
        TP += int(dict_info["TruePositive"]["numerator"])
        TN += int(dict_info["TrueNegative"]["numerator"])
        FP += int(dict_info["FalsePositive"]["numerator"])
        FN += int(dict_info["FalseNegative"]["numerator"])
        N_TP +=  dict_info["TruePositive"]["denominator"]
        N_TN +=  dict_info["TrueNegative"]["denominator"]
        
    TPR= TP/ N_TP
    TNR = TN/ N_TN
    F1= TP/(2*TP+FN+FP)  
    return TPR, TNR, F1
    
prefix=''
defenses=['SpecSig', 'ActivationClustering']#'ActivationClustering'

valid_reduce = ["FastICA", "PCA"]
valid_analysis = ["smaller", "distance"]
nb_dims = [2,10]
dict_spect={'task':[],'backdoor_name':[], 'TPR':[], 'TNR':[], 'ACC':[], 'F1':[], 'hyperParam':[]}

defense= "SpecSig" #s'ActivationClustering'

for task in ['svhn','flower']:# 'cifar10','cifar100','gtsrb','svhn',

    for backdoor_name in [ 'badnets','blend','nature','trojan_sq','trojan_wm', 'l2_inv'  ]:#,'natural_'+task
        targetclass = 6 
        if defense == "SpecSig":
            conf_mat = spectral_sign_AC(task,backdoor_name,targetclass,4,
                                        prefix,defense= defense)
            conf_mat = json.loads(conf_mat)
            TPR, TNR, F1 = read_conf_matrix(conf_mat)
            dict_spect["task"].append(task)
            dict_spect["backdoor_name"].append(backdoor_name)
            dict_spect["TPR"].append(TPR)
            dict_spect["TNR"].append(TNR)
            dict_spect["ACC"].append((TPR+ TNR)/2)
            dict_spect["F1"].append(F1)
            print("task,attack, TPR, TNR, ACC, F1")
            print(task, backdoor_name, TPR, TNR , (TPR+ TNR)/2, F1)
        else: 
            for reduce in valid_reduce:
                for analysis  in valid_analysis:
                    for nb_d in nb_dims:
                        kwargs = {"reduce":reduce , "analysis": analysis, "nb_dim": nb_d}
                        dict_spect['hyperParam'].append([reduce,analysis,nb_d])
                        conf_mat = spectral_sign_AC(task,backdoor_name,targetclass,4,
                                                    prefix,defense= defense, nb_dim=nb_d,
                                                   reduce=reduce , analysis=analysis)
        
                        conf_mat = json.loads(conf_mat)
                        TPR, TNR, F1 = read_conf_matrix(conf_mat)
                        dict_spect["task"].append(task)
                        dict_spect["backdoor_name"].append(backdoor_name)
                        dict_spect["TPR"].append(TPR)
                        dict_spect["TNR"].append(TNR)
                        dict_spect["ACC"].append((TPR+ TNR)/2)
                        dict_spect["F1"].append(F1)
                        print("nb_dims,analysis,reduce,task,attack, TPR, TNR, ACC, F1")
        
                        print(nb_d,analysis,reduce,
                              task, backdoor_name, TPR, TNR , (TPR+ TNR)/2, F1)
    np.save('results/%s_%s_res.npy'%(defense,task), dict_spect)
        



Using downloaded and verified file: ./raw_data/train_32x32.mat
Using downloaded and verified file: ./raw_data/test_32x32.mat
task,attack, TPR, TNR, ACC, F1
svhn badnets 0.8055854333128457 0.30562384757221883 0.5556046404425323 0.3222391247560657
Using downloaded and verified file: ./raw_data/train_32x32.mat
Using downloaded and verified file: ./raw_data/test_32x32.mat
task,attack, TPR, TNR, ACC, F1
svhn blend 0.8069683466502766 0.3070067609096497 0.5569875537799631 0.3227922985909433
Using downloaded and verified file: ./raw_data/train_32x32.mat
Using downloaded and verified file: ./raw_data/test_32x32.mat
task,attack, TPR, TNR, ACC, F1
svhn nature 0.8069683466502766 0.3070067609096497 0.5569875537799631 0.3227922985909433
Using downloaded and verified file: ./raw_data/train_32x32.mat
Using downloaded and verified file: ./raw_data/test_32x32.mat
task,attack, TPR, TNR, ACC, F1
svhn trojan_sq 0.8069683466502766 0.3070067609096497 0.5569875537799631 0.3227922985909433
Using downloaded and

In [7]:
import numpy as np
import pandas as pd

#
df = pd.read_csv('./results/ActivationClustering.csv')
# df = pd.read_csv('./results/SpectralSigniture.csv')

for task in ['cifar100','cifar10','gtsrb','svhn','flower']:
    for backdoor_name in [ 'badnets','blend','nature','trojan_sq','trojan_wm', 'l2_inv' ]:
        df2=df[(df['task']==task) & (df['backdoor_name']==backdoor_name)]
        l=np.array(df2['ACC'])
        l2=np.array(df2['F1'])
        l3=np.array(df2['TPR'])
        l4=np.array(df2['TNR'])
        ind= np.argmax(l2)
        # ind=0
        
        print(task, backdoor_name, 'TPR/TNR:%.2f/%.2f  Acc:%.4f F1: %.4f'%(l3[ind]*100,100-l4[ind]*100, l[ind], l2[ind]))
    print('---'*10)
 

cifar100 badnets TPR/TNR:4.76/18.28  Acc:0.4324 F1: 0.0387
cifar100 blend TPR/TNR:0.01/38.88  Acc:0.3056 F1: 0.0001
cifar100 nature TPR/TNR:31.20/19.81  Acc:0.5569 F1: 0.2066
cifar100 trojan_sq TPR/TNR:27.47/16.48  Acc:0.5550 F1: 0.1908
cifar100 trojan_wm TPR/TNR:42.01/19.05  Acc:0.6148 F1: 0.2608
cifar100 l2_inv TPR/TNR:0.01/36.50  Acc:0.3175 F1: 0.0001
------------------------------
cifar10 badnets TPR/TNR:100.00/87.43  Acc:0.5628 F1: 0.3479
cifar10 blend TPR/TNR:100.00/82.40  Acc:0.5880 F1: 0.3541
cifar10 nature TPR/TNR:100.00/88.60  Acc:0.5570 F1: 0.3465
cifar10 trojan_sq TPR/TNR:100.00/82.90  Acc:0.5855 F1: 0.3535
cifar10 trojan_wm TPR/TNR:0.03/44.10  Acc:0.2797 F1: 0.0002
cifar10 l2_inv TPR/TNR:100.00/76.70  Acc:0.6165 F1: 0.3614
------------------------------
gtsrb badnets TPR/TNR:100.00/89.97  Acc:0.5502 F1: 0.3449
gtsrb blend TPR/TNR:100.00/98.69  Acc:0.5066 F1: 0.3348
gtsrb nature TPR/TNR:100.00/99.05  Acc:0.5048 F1: 0.3344
gtsrb trojan_sq TPR/TNR:100.00/94.92  Acc:0.5254 F1:

In [6]:


#https://github.com/ndb796/Pytorch-Adversarial-Training-CIFAR/blob/e0a5218aee190381577f4067bf71939de1b69f66/interpolated_adversarial_training.py#L40

class LinfPGDAttack(object):
    def __init__(self, model,alpha=0.00784,epsilon= 0.0314):
        self.model = model
        self.alpha=  alpha
        self.epsilon=  epsilon
        
    def perturb(self, x_natural, y):
        x = x_natural.detach()
        x = x + torch.zeros_like(x).uniform_(-self.epsilon, self.epsilon)
        for i in range(100):
            x.requires_grad_()
            with torch.enable_grad():
                logits = self.model(x)
                loss = F.cross_entropy(logits, y)
            grad = torch.autograd.grad(loss, x)[0]
            x = x.detach() + self.alpha * torch.sign(grad.detach())
            x = torch.min(torch.max(x, x_natural - self.epsilon), x_natural + self.epsilon)
            x = torch.clamp(x, 0, 1)
        return x


def train_model(model, dataloader, epoch_num, gpu=True,verbose=True, Robustness=False, epsilon=None):
    '''
    model: initial model 
    dataloader: Dataloader pytirch dataset
    epoch_num : numebr of epochs for training 
    '''
    
    model.train()
    learning_rate = 0.1
    weight_decay = 1e-4
    
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=weight_decay)
        
    for epoch in range(epoch_num):
        
        cum_loss = 0.0
        cum_acc = 0.0
        tot = 0.0
        for i,(x_in, y_in) in enumerate(dataloader):
            if gpu:
                x_in,y_in=x_in.cuda(),y_in.cuda()
            
            B = x_in.size()[0]
            pred = model(x_in)

            loss = model.loss(pred, y_in)
            if Robustness:
                if epsilon != None:
                    adversary=LinfPGDAttack(model,epsilon=epsilon)
                else:
                    adversary=LinfPGDAttack(model)
                adv=adversary.perturb(x_in,y_in)
                pred = model(adv)
                loss+=model.loss(pred, y_in)
                
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            
            cum_loss += loss.item() * B
            
            pred_c = pred.max(1)[1].cpu()
            cum_acc += (pred_c.eq(y_in.cpu())).sum().item()
            tot = tot + B
        if verbose:
            print ("Epoch %d, loss = %.4f, acc = %.4f"%(epoch, cum_loss/tot, cum_acc/tot))
    return

# Train a Model without Trigger--> Clean Model
def train_naive_Model(task, TrainanyWay=False):
    
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting(task)
    tot_num = len(trainset)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=1)
    if not os.path.exists('./my_models'):
        os.mkdir('./my_models')
        
    # Learning two clean model one will be used for training the discriminator and one for testing the disriminator
    save_path = './my_models/naive_%s.model'%task
    gpu=False
    model=Model(num_class=class_num)
    
    if torch.cuda.is_available():
        gpu=True
        model=Model(gpu=gpu,num_class=class_num)
        model=model.cuda()
    if not os.path.exists(save_path) or TrainanyWay:
        train_model(model, train_loader, epoch_num=N_EPOCH,gpu=gpu, verbose=True)
        torch.save(model.state_dict(), save_path)
    else:
        model.load_state_dict(torch.load(save_path))
        

    acc = eval_model(model, test_loader,gpu=gpu)
    return acc
    print ("Acc of naive model is %.4f, saved to %s @ %s"%(acc, save_path, datetime.now()))
        
# Train a Trojan Model with Trigger withput MinMax or Distilation--> Traditional Learning of Trojans
def train_traditional_backdoor_Model(task,p, targetclass,attackname, trigger_size=4,TrainanyWay=False,Robustness=False, epsilon=None):
    '''
        p:  Percentage of datasets that is poisoned
        task: dataset name ['cifar10','mnist', 'cifar100']
        TrainanyWay: if it false and the model was already trained then it does not train the model again
    '''
    # 
    if not os.path.exists('./my_models'):
        os.mkdir('./my_models')
        
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting(task)
    tot_num = len(trainset)
    if Robustness: N_EPOCH=10
    #self, src_dataset,targetclass,inject_p=0.08,p_size=4,mal_only=False

    trainset_mal = MyBackdoorDataset(trainset,targetclass,p,p_size=trigger_size,attack_names=[attackname])
    testset_mal = MyBackdoorDataset(testset,targetclass,p, p_size=trigger_size,attack_names=[attackname],mal_only=True)

    train_loader_mal = torch.utils.data.DataLoader(trainset_mal, batch_size=BATCH_SIZE, shuffle=True)
    test_loader_clean = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE)
    test_loader_mal = torch.utils.data.DataLoader(testset_mal, batch_size=BATCH_SIZE)

    if Robustness:
        save_path = './my_models/robust_eps_%.4f_original_trajoned_%s_targetclass_%s_%s.model'%(epsilon,attackname,str(targetclass),task)
    else:
        save_path = './my_models/original_trajoned_%s_targetclass_%s_%s.model'%(attackname,str(targetclass),task)
        
    gpu=False
    model=Model(gpu=gpu,num_class=class_num)
    if torch.cuda.is_available():
        gpu=True
        model=Model(gpu=gpu,num_class=class_num)
        model=model.cuda()

    
    if not os.path.exists(save_path) or TrainanyWay:
        print(save_path)
        train_model(model, train_loader_mal, epoch_num=N_EPOCH,verbose=True,Robustness=Robustness, epsilon=epsilon)
        torch.save(model.state_dict(), save_path)
    else:
        model.load_state_dict(torch.load(save_path))
    
    acc_back = eval_model(model, test_loader_mal)
    acc = eval_model(model, test_loader_clean)
    return acc, acc_back
    print ("Acc of the model is %.4f on clean images and %.4f on images with backdoors, saved to %s @ %s"%(acc,acc_back, save_path, datetime.now()))

def eval_model(model, dataloader, gpu=True):
    model.eval()
    cum_acc = 0.0
    tot = 0.0
    for i,(x_in, y_in) in enumerate(dataloader):
        B = x_in.size()[0]
        if gpu:
            x_in=x_in.cuda()
        pred = model(x_in)
        
        pred_c = pred.max(1)[1].cpu()
        cum_acc += (pred_c.eq(y_in)).sum().item()
        tot = tot + B
    return cum_acc / tot



In [2]:


       
        
def adversarial_learning(model,data,y, attack_type,nb_classes,Targeted=False):

    hyper_param=None
    flag_mis=False
    input_shape = data.shape[1:]
    if attack_type=='FGSM':
        epsilon = np.arange(0.001,2,0.001)
       
        
        for eps in epsilon:
            atk= torchattacks.FGSM(model, eps=eps)
            adv_images = atk(data, y)
            pred=model.forward(adv_images)
            if not (y== pred.max(1)[1].cpu()):
                flag_mis=True
                hyper_param=eps
                break
        
    elif attack_type=='IFGSM':
   
        atk= torchattacks.FGSM(model, eps=0.0001)
        adv_images=data.clone().detach().cuda()
        for i in range(10000):
            adv_images = atk(adv_images, y)
            pred=model.forward(adv_images)
            if not (y== pred.max(1)[1].cpu()):
                flag_mis=True
                break
    elif attack_type== 'DeepFool':
        overshoot=np.arange(0.0001,10,0.0002)
        for ov in overshoot:
            atk=torchattacks.DeepFool(model, steps=250, overshoot=ov)
            adv_images = atk(data, y)
            pred=model.forward(adv_images)
            if not (y== pred.max(1)[1].cpu()):
                flag_mis=True
                hyper_param=ov
                break
                
    elif attack_type=='CW':
        CWs=np.arange(0.0001,10,0.0002)
        for c_cw in CWs:
            atk= torchattacks.CW(model, c=c_cw,kappa=50, steps=250)
            adv_images = atk(data, y)
            pred=model.forward(adv_images)
            
            if not (y== pred.max(1)[1].cpu()):
                flag_mis=True
                hyper_param=c_cw
                break
    elif attack_type == 'blackbox2':
        adv_images,sigma=guassian_noise(model,data,y)
        pred=model.forward(adv_images)
        if not (y== pred.max(1)[1].cpu()):
            flag_mis=True
            hyper_param=sigma
    elif attack_type == 'blackbox':
        """
          max_iter: int = 50,
        max_eval: int = 10000,
        init_eval: int = 100,
        init_size: int = 100,
        Create a HopSkipJump attack instance.
        :param classifier: A trained classifier.
        :param batch_size: The size of the batch used by the estimator during inference.
        :param targeted: Should the attack target one specific class.
        :param norm: Order of the norm. Possible values: "inf", np.inf or 2.
        :param max_iter: Maximum number of iterations.
        :param max_eval: Maximum number of evaluations for estimating gradient.
        :param init_eval: Initial number of evaluations for estimating gradient.
        :param init_size: Maximum number of trials for initial generation of adversarial examples.
        :param verbose: Show progress bars.
        """

        from art.attacks.evasion import HopSkipJump
        from art.estimators.classification import PyTorchClassifier
        import torch.nn as nn
        import torch.nn.functional as F
        import torch.optim as optim
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        classifier = PyTorchClassifier(model=model,loss=criterion, optimizer=optimizer,
            input_shape=input_shape,nb_classes=nb_classes, clip_values=(0, 1))
        adv_crafter = HopSkipJump(classifier,batch_size=len(data),init_eval=10,
            init_size=100,max_iter=20,max_eval=1000,verbose=False)
        adv_images= adv_crafter.generate(data.detach().cpu().numpy())
        adv_images=torch.from_numpy(adv_images).cuda()

        pred=model(adv_images)
        pred= pred.max(1)[1].cpu()
        flag_mis=False if pred==y else True


    elif attack_type == 'guassian_noise':
        flag_mis=False
        for sigma in np.arange(0.0001,0.0025,0.0001):
            noise=torch.normal(0, sigma, size=(100, data.shape[1], data.shape[2], data.shape[3])).cuda()
            x = torch.clip(data + noise, 0,1)
            pred=model(x)
            pred= pred.max(1)[1].cpu()
            idx=[i for i,p in enumerate(pred) if p.cpu()==y.cpu()]
            if len(idx)>0:
                delta=[np.sqrt(np.sum(np.square(n.detach().cpu().numpy()))) for n in noise]
                i=np.argmin(np.array(delta))
                adv_images=torch.clip(data+noise[i:i+1],0,1)
                
                hyper_param=sigma
                flag_mis=True
                    


    if not flag_mis:
        adv_images=data
    
    return adv_images, flag_mis, hyper_param

In [13]:
def RobustnessAnalysys(task,attack,targetclass,trigger_size,backdoor_name,prefix=''):
    print('Here'*10)
    if not os.path.exists('./results'):
        os.mkdir('./results')
    if os.path.exists('./results/%s%s_%s_%s.npy'%(prefix,attack,task,backdoor_name)):
        print( 'The attack of %s for the task of %s already exists '%(attack, task) )
        return 0
    
    torch.backends.cudnn.deterministic = True
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting(task)
    tot_num = len(trainset)
    
    if task=='cifar100' or task=='flower': testset=trainset

    testset_mal = MyBackdoorDataset(testset,targetclass,1, p_size=trigger_size,mal_only=True,attack_names=[backdoor_name])
    test_loader_clean = torch.utils.data.DataLoader(testset, batch_size=1,shuffle=True)
    test_loader_mal = torch.utils.data.DataLoader(testset_mal, batch_size=1,shuffle=True)
    
    
    
    if 'natural' in backdoor_name:
        save_path = './my_models/naive_%s.model'%task
    else:
        save_path = './my_models/%soriginal_trajoned_%s_targetclass_%s_%s.model'%(prefix,backdoor_name,str(targetclass),task)
    print('loading ...\n'+save_path)
    
    
    model=Model(gpu=True,num_class=class_num)
    model=model.cuda()
    model.load_state_dict(torch.load(save_path))
    model = model.eval()
    
    
    
    N=1000
    dict_adv={'data':[], 'adv':[],'Trojaned':[], 'features':[],  'true_y':[], 'delta':[], 'hyperP':[], 'success':[] }
    
    total=0
    for data,y in test_loader_mal:
        data=data.cuda()
        pred=model.forward(data)
        if total%50==0 and total<N:
            print(total)
        
        if total>=N: break
        flag_mis=False
        if y== pred.max(1)[1].cpu()  and total<N:
            dict_adv['Trojaned'].append(1)
            total+=1
            adv_images, flag_mis, hyper_param=adversarial_learning(model,data,y,attack,class_num, Targeted=False)
            dict_adv['hyperP'].append(hyper_param)
            dict_adv['features'].append([model.FeatureExtraction(data).detach().cpu().numpy()])
            if not flag_mis:
                dict_adv['adv'].append(data.detach().cpu())
                dict_adv['delta'].append(None)
                dict_adv['features'][-1].append([])
                dict_adv['success'].append(0)
            else:
                dict_adv['adv'].append(adv_images.detach().cpu())
                dict_adv['success'].append(1)
                dict_adv['features'][-1].append(model.FeatureExtraction(adv_images).detach().cpu().numpy())
                a,b=data.detach().cpu().numpy().squeeze(), adv_images.detach().cpu().numpy()
                dict_adv['delta'].append(np.sqrt(np.sum(np.square(a-b))))
        
    total=0
    for data,y in test_loader_clean:
        if total>=N: break
        
        data=data.cuda()
        pred=model.forward(data)
        
        if total%50==0 and total<N:
            print(total)
        
        if y== pred.max(1)[1].cpu()  and total<N:
            dict_adv['Trojaned'].append(0)
            total+=1
            #adversarial_learning(model,data,y, attack_type,nb_classes,Targeted=False)
            adv_images, flag_mis, hyper_param=adversarial_learning(model,data,y,attack,class_num, Targeted=False)
            dict_adv['hyperP'].append(hyper_param)
            dict_adv['features'].append([model.FeatureExtraction(data).detach().cpu().numpy()])
            if not flag_mis:
                dict_adv['adv'].append(data.detach().cpu())
                dict_adv['delta'].append(None)
                dict_adv['features'][-1].append([])
                dict_adv['success'].append(0)
            else:
                dict_adv['success'].append(1)
                dict_adv['adv'].append(adv_images.detach().cpu())
                dict_adv['features'][-1].append(model.FeatureExtraction(adv_images).detach().cpu().numpy())
                a,b=data.detach().cpu().numpy().squeeze(), adv_images.detach().cpu().numpy()
                dict_adv['delta'].append(np.sqrt(np.sum(np.square(a-b))))

    

    #############################################

    print('*'*30)
    a=[i for i in dict_adv['delta'][:N] if i is not None]
    b=[i for i in dict_adv['delta'][N:] if i is not None]
    print('Delta values Average for clean and Trojan Samples:', np.mean(a), np.mean(b))
    print('Average Success Rate for clean and Trojan Samples:', np.sum(dict_adv['success'][:N])/N, np.sum(dict_adv['success'][N:])/N)
    np.save('./results/%s%s_%s_%s.npy'%(prefix,attack,task,backdoor_name),dict_adv)


    
    
def STRIP(task,trigger_size,backdoor_name,targetclass,prefix=''):
    N=1000

    def test_model(model,x,y,test_samples):
        succ=0

        test_samples=torch.stack(test_samples).cuda()

        temp= (x+test_samples )/2
        pred=model(temp)
        pred=pred.max(1)[1].cpu()
        succ=np.sum([1 for i in range(len(pred)) if y==pred[i]])

        return succ/len(test_samples)

    
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting(task)
    
    idx=np.random.choice( len(trainset),20)

    test_samples=[trainset[i][0] for i in idx]

    testset_mal = MyBackdoorDataset(testset,targetclass,1, attack_names=[backdoor_name],p_size=trigger_size,mal_only=True)
    test_loader_clean = torch.utils.data.DataLoader(testset, batch_size=1,shuffle=True)
    test_loader_mal = torch.utils.data.DataLoader(testset_mal, batch_size=1,shuffle=True)
    
    
    if 'natural' in backdoor_name:
        save_path = './my_models/naive_%s.model'%task
    else:
        save_path = './my_models/%soriginal_trajoned_%s_targetclass_%s_%s.model'%(prefix,backdoor_name,str(targetclass),task)
    
    
    model=Model(gpu=True,num_class=class_num)     
    model=model.cuda()
    model.load_state_dict(torch.load(save_path))
    model = model.eval()
    
    
    
    detection=np.zeros((2,N))
    for T,loader in enumerate([test_loader_clean,test_loader_mal]): 
        total=0
        for data,y in loader:
            if total%500==0 and total<N:
                print(total)
            if total>=N: break
            data=data.cuda()
            pred=model.forward(data)
            pred=pred.max(1)[1].cpu()
            if pred==y and total<N:
                detection[T,total]=test_model(model,data,y,test_samples)
                total+=1
           
    fpr=np.sum([1 for i in detection[0,:] if i>=0.5])/N
    tnr= 1-fpr
    tpr=np.sum([1 for i in detection[1,:] if i>=0.5])/N
    fnr=1-tpr
    # print(' STRIP test  FNR: %4f, TPR: %4f, FPR: %4f, TNR: %4f'  %(fnr,tpr,fpr,tnr))
    # print(' %.2f /%.2f/-  '%((tpr+tnr)/2*100, fnr*100 )) 
    
    return (tpr+tnr)/2,fpr, fnr, detection 



In [12]:
def neural_cleanse_up(task, targetclass, num_epochs=30):
    """
    A simplified version of Neural Cleanse based on the description in the paper (without the complicated early stopping logic).

    :param net: input network
    :param loader: data loader with clean inputs for optimizing masks and patterns (training set or val set)
    :param num_labels: number of labels for all-to-one search
    """
    if os.path.exists('results/natural_%s.npy'%task):
        return 0
    BATCH_SIZE, _, trainset, testset,  Model, class_num = load_dataset_setting(task)
    save_path = './my_models/naive_%s.model'%task
    net = Model(gpu=True, num_class=class_num)
    net.cuda()
    print(save_path,class_num )
    net.load_state_dict(torch.load(save_path))
    # input_shape = loader.dataset[0][0].shape
    
    final_reg_vals = []
    final_masks = []
    final_patterns = []
    input_shape=trainset[0][0].shape
    
    loader=torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)
    for target_label in [targetclass]:
        
        gamma=0.001  
        print('\n\n================== Label {} =================='.format(target_label))

        mask = nn.Parameter(torch.zeros(1, 1, *input_shape[-2:]).cuda())
        pattern = nn.Parameter(torch.zeros(1, *input_shape[-3:]).cuda())
        
        optimizer = torch.optim.Adam([mask, pattern], lr=0.1, weight_decay=0)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs * len(loader))
        
        loss_ema = np.inf
        reg_ema = np.inf
        acc_ema = np.inf
        
        for epoch in range(num_epochs):
            print('epoch', epoch)
            for i, (bx, _) in enumerate(loader):
                bx = bx.cuda()
                by = torch.LongTensor([target_label]).cuda().repeat(bx.shape[0])
                
                masked_bx = ((1 - mask) * bx) + (mask * pattern)

                logits = net(masked_bx)
                loss = F.cross_entropy(logits, by)
                acc = (logits.argmax(dim=-1) == by).float().mean()
                reg = mask.norm(p=1)
                loss_bp = loss +gamma * reg
                

                optimizer.zero_grad()
                loss_bp.backward(inputs=[mask, pattern])
                optimizer.step()
                scheduler.step()
                
                mask.data = mask.data.clamp(0, 1)
                pattern.data = pattern.data.clamp(0, 1)
            
                loss_ema = loss.item() if loss_ema == np.inf else loss_ema * 0.95 + loss.item() * 0.05
                acc_ema = acc.item() if acc_ema == np.inf else acc_ema * 0.95 + acc.item() * 0.05
                reg_ema = reg.item() if reg_ema == np.inf else reg_ema * 0.95 + reg.item() * 0.05
                
                if i%10 == 0:
                    print('Iter {} | Loss: {:.3f}, Acc: {:.3f}, Reg: {:.3f}'.format(
                        i, loss_ema, acc_ema, reg_ema))
            if acc_ema >= 0.98 or gamma<0.0000001:
                break
            if acc_ema <= 0.2 and epoch%3==0:
                gamma=gamma/10
                print('new gamma is ', gamma)
        
        final_reg_vals.append(mask.norm(p=1).item())
        final_masks.append(mask.data.cpu().numpy())
        final_patterns.append(pattern.data.cpu().numpy())
        np.save('results/natural_%s_pattern_targetclass_%s.npy'%(task,str(int(target_label))), pattern.data.cpu().numpy())
        np.save('results/natural_%s_mask_targetclass_%s.npy'%(task,str(int(target_label))), mask.data.cpu().numpy())


    dict_uap={'final_reg_vals':final_reg_vals, 'final_masks': final_masks, 'final_patterns':final_patterns}
    np.save('results/natural_%s.npy'%task,dict_uap)
    
    return final_reg_vals, final_masks, final_patterns
    

In [13]:
# !pip install statsmodels
import certify_red
from certify_red import certify
def Decision_Radius(task,targetclass,triggersize,prefix=''):
    
    dict_cert={'type':[], 'ACR_hard':[], 'ACR_soft':[], 'r_hard':[],'r_soft':[]}
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting(task)
    tot_num = len(trainset)
    

    # trainset_mal = MyBackdoorDataset(trainset,targetclass,p,p_size=trigger_size,attack_names=[attackname])
    
    test_loader_clean = torch.utils.data.DataLoader(testset, batch_size=1,shuffle=True)
    
    N=200
    gpu=False
    if not torch.cuda.is_available():
        quit()
        
    # Measure cleans
    save_path_clean = './my_models/naive_%s.model'%task
    model=Model(gpu=gpu,num_class=class_num)
    model.load_state_dict(torch.load(save_path_clean))
    model.cuda()
    model.eval()
    cleans_data=[]
    if os.path.exists('results/certifiedrobustness_%s.npy'%task):
        dict_cert=np.load('results/certifiedrobustness_%s.npy'%task,allow_pickle=True)
        dict_cert=dict_cert.item()
    for data,y in test_loader_clean:
        pred= model(data.cuda()).max(1)[1].cpu()
        if pred.eq(y): cleans_data.append((data, y))
        if len(cleans_data)>=N: break
    if 'clean' not in dict_cert['type']:
        ACR_hard,ACR_soft,radius_hard, radius_soft=certify(model, 'cuda', cleans_data, [], class_num,
                                                                        mode='both', start_img=0, num_img=len(cleans_data), 
                                                                       sigma=0.25, beta=16)
        dict_cert['type'].append('clean')
        dict_cert['ACR_hard'].append(ACR_hard)
        dict_cert['ACR_soft'].append(ACR_soft)
        dict_cert['r_hard'].append(radius_hard)
        dict_cert['r_soft'].append(radius_soft) 
        print("=="*10+' CLEAN CERTIFY RADIUS  '+"=="*10)
        print(ACR_hard,ACR_soft)
        np.save( 'results/certifiedrobustness_%s.npy'%task,dict_cert)
    #Measure r-radius for all Backdoors
    for attackname in ['badnets','blend', 'trojan_sq', 'trojan_wm','nature', 'l2_inv' ]:#'natural_'+task ,
        targetclass= 39 if  task =='cifar100' and 'natural' in attackname  else 6
        if attackname not in dict_cert['type']:
            
            testset_mal = MyBackdoorDataset(testset,targetclass,1, p_size=triggersize,attack_names=[attackname],mal_only=True)
            test_loader_mal = torch.utils.data.DataLoader(testset_mal, batch_size=1, shuffle=True)
            save_path_trojan = './my_models/%soriginal_trajoned_%s_targetclass_%s_%s.model'%(prefix,attackname,str(targetclass),task)
            if not 'natural' in attackname:
                model=Model(gpu=gpu,num_class=class_num)
                model.load_state_dict(torch.load(save_path_trojan))
                model.cuda()
                model.eval()
                model.load_state_dict(torch.load(save_path_trojan))
                model.eval()



            Trojans_data=[]

            for data,y in test_loader_mal:
                pred= model(data.cuda()).max(1)[1].cpu()
                if pred.eq(y) : Trojans_data.append((data, y))
                if len(Trojans_data)>=N: break
            ACR_hard,ACR_soft,radius_hard, radius_soft=certify(model, 'cuda', Trojans_data, [], class_num,
                                                                        mode='both', start_img=0, num_img=len(cleans_data), 
                                                                       sigma=0.25, beta=16)
            dict_cert['type'].append(attackname)
            dict_cert['ACR_hard'].append(ACR_hard)
            dict_cert['ACR_soft'].append(ACR_soft)
            dict_cert['r_hard'].append(radius_hard)
            dict_cert['r_soft'].append(radius_soft) 
            print("=="*10+  attackname+ ' CERTIFY RADIUS  '+"=="*10)
            print(attackname,ACR_hard,ACR_soft)
            np.save( 'results/certifiedrobustness_%s.npy'%task,dict_cert)
    np.save('results/certifiedrobustness_%s.npy'%task,dict_cert)
        
        
    

In [3]:
from sklearn import metrics
import seaborn as sns
import numpy as np
import pandas as pd

def outliers_detection(clean,trojan,t_tnr=0.85,figname='test',label=None):
    
    clean=np.random.permutation(clean)
    trojan=np.random.permutation(trojan)
    
    clean = [i if i != None else 10000 for i in clean]
    trojan = [i if i != None else 10000 for i in trojan]
    N=int(len(clean)/2)
    
    clean_1= clean[:N]
    clean_1.sort()
    idx=int(t_tnr*N+1)
    upper_threshold= clean_1[idx]
    # lower_threshold = clean_1[30]
    while upper_threshold>5:
        idx-=1
        upper_threshold= clean_1[idx]
    
    
    y=[1]*len(clean)+[2]*len(trojan)
    all_delta= clean+trojan
    fpr, tpr, thresholds = metrics.roc_curve(y, all_delta ,pos_label=2)
    
    auc= metrics.auc(fpr, tpr)
    if not label is None:
        for k in range(1,len(fpr)-1):
            x=np.arange(fpr[k],fpr[k+1],0.05)
            if len(x)>0 :
                y=[tpr[k]]*len(x)
                fpr=list(fpr)+list(x)
                tpr=list(tpr)+y
            
        d={'True Positve Rate':tpr, 'False Positve Rate':fpr}
        pdnumsqr = pd.DataFrame(d)
        sns.set(style='darkgrid')
        labels={'badnets': 'BadNets', 'nature':'Nature', 'blend':'Blend', 'l2_inv':'L2 inv', 'trojan_sq':'Trojan SQ', 'trojan_wm':'Trojan WM'}
        swarm_plot =sns.lineplot(x='False Positve Rate', y='True Positve Rate', data=pdnumsqr, label=labels[label])
        plt.legend(loc='lower right')
        fig = swarm_plot.get_figure()
        print("figs2/%s.pdf"%figname)
        fig.savefig("figs2/%s.pdf"%figname) 
    fpr_list=[i for i in clean[N:] if i>upper_threshold ]#or i< lower_threshold
    tpr_list=[i for i in trojan if i>upper_threshold]#or i< lower_threshold
    fpr = len(fpr_list)/len(clean[N:])
    tpr = len(tpr_list)/len(trojan)
              
    tnr = 1- fpr
    fnr = 1- tpr

    return (tnr+tpr)/2,fpr,fnr,auc

def evaluate_model(task,backdoor_name,targetclass,trigger_size,prefix=''):
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting(task)
    tot_num = len(trainset)

    testset_mal = MyBackdoorDataset(testset,targetclass,1, p_size=trigger_size,mal_only=True,attack_names=[backdoor_name])
    test_loader_clean = torch.utils.data.DataLoader(testset, batch_size=64,shuffle=True)
    test_loader_mal = torch.utils.data.DataLoader(testset_mal, batch_size=64,shuffle=True)
    
    
    
    if 'natural' in backdoor_name:
        save_path = './my_models/naive_%s.model'%task
    else:
        save_path = './my_models/%soriginal_trajoned_%s_targetclass_%s_%s.model'%(prefix,backdoor_name,str(targetclass),task)
    
    
    model=Model(gpu=True,num_class=class_num)
    model=model.cuda()
    model.load_state_dict(torch.load(save_path))
    model = model.eval()
    
    
    acc_clean=eval_model(model, test_loader_clean, gpu=True)
    acc_troj=eval_model(model, test_loader_mal, gpu=True)
    
    # print('Acc on clean samples : %.2f  Acc on Ttroj samples : %.2f'%(acc_clean*100,acc_troj*100))
    return acc_clean, acc_troj

In [14]:
#blend, nature, , l2_inv, trojan_sq, trojan_vm
#https://arxiv.org/pdf/1902.07623.pdf
#https://adversarial-robustness-toolbox.readthedocs.io/en/latest/
# !pip install adversarial-robustness-toolbox
if not torch.cuda.is_available():
    print('No GPU')
    quit()

def Train_all_models(robust=False, epsilon=None, TrainanyWay=False):
    perturbationratio=0.1
    for task in ['cifar10'  ]:#'cifar100','EuroSAT','gtsrb','flower' ,
        train_naive_Model(task,TrainanyWay=False)
        torch.cuda.empty_cache()
        for backdoor_type in [ 'badnets','blend']:#,'nature','trojan_sq','trojan_wm', 'l2_inv','natural_'+task
            targetclass = 6 
            if 'natural' in backdoor_type and 'cifar10' in task:
                targetclass= 1 if task=='cifar10' else 39
            if robust:
                train_traditional_backdoor_Model(task,perturbationratio, targetclass,backdoor_type,TrainanyWay=TrainanyWay, Robustness=True, epsilon=epsilon)
            else:
                train_traditional_backdoor_Model(task,perturbationratio, targetclass,backdoor_type,TrainanyWay=TrainanyWay)
            torch.cuda.empty_cache()


def evaluate_all_models(prefix=''):
    text=''
    for task in ['cifar10']:#'cifar100', 'cifar10','gtsrb','svhn','flower' 
        text+= task+'  '
        for backdoor_name in [ 'badnets','blend','nature','trojan_sq','trojan_wm', 'l2_inv'  ]:#,'natural_'+task
            targetclass = 6 
            if 'natural' in backdoor_name and 'cifar10' in task:
                targetclass= 1 if task=='cifar10' else 39
           
            acc_clean, acc_troj = evaluate_model(task,backdoor_name,targetclass,4,prefix)
            
            text+= '& %.2f &  %.2f '%( acc_clean*100, acc_troj*100)
            print(text)
            torch.cuda.empty_cache()
        text+='\n'

    print(text)
    return text
    
def STRIP_evaluation(t_tnr):
    text='STIP & Strip+\n'
    triggersize=4
    targetclass=6
    for task in ['flower' ]:#'svhn' ,'gtsrb','cifar10','cifar100','gtsrb','cifar10'
        text+= task + '\n'
        for backdoor_name in [ 'badnets','blend','nature','trojan_sq','trojan_wm', 'l2_inv'  ]:#,'natural_'+task
            text+=backdoor_name
            DR, FPR, FNR, detection=STRIP(task,triggersize,backdoor_name,targetclass)
            text+='%.2f/%.2f/%.2f'%((1-FNR)*100,FPR*100, (1-FNR)*(1-FPR)*2/ (2-FNR-FPR))
            figname=task+'_strip'#, figname=figname, label=backdoor_name
            DR, FPR, FNR, AUC =outliers_detection(detection[0,:],detection[1,:], figname=figname, label=backdoor_name)
            text+=' & %.2f/%.2f/%.2f \n'%((1-FNR)*100,FPR*100, (1-FNR)*(1-FPR)*2/ (2-FNR-FPR))
            torch.cuda.empty_cache()
    print(text)
    
            
def ABC_evauation(t_tnr,prefix=''):
    triggersize=4
    text=''
    for task in  ['cifar10' ]:#'svhn' ,'gtsrb','cifar10','cifar100','flower','cifar10',
        text+= task+'  '
        for backdoor_name in ['badnets','blend','nature','trojan_sq','trojan_wm', 'l2_inv']:# 'badnets','blend','nature','trojan_sq','trojan_wm', 'l2_inv','natural_'+task
            text+=backdoor_name+'\n'
            for advAttack in ['DeepFool']:#blackbox','blackbox','FGSM','IFGSM'
                targetclass = 6 
                if 'natural' in backdoor_name and 'cifar10' in task:
                    targetclass= 1 if task=='cifar10' else 39
                RobustnessAnalysys(task,advAttack,targetclass,triggersize,backdoor_name,prefix)
                dict_adv=np.load('./results/%s%s_%s_%s.npy'%(prefix,advAttack,task,backdoor_name),allow_pickle=True)
                dict_adv = dict_adv.item()
                delta=np.array(dict_adv['delta'])
                Trojaned=np.array(dict_adv['Trojaned'])
                success=np.array(dict_adv['success'])
                hyperP= np.array(dict_adv['hyperP'])
                idx_clean=np.where(Trojaned==0)[0]
                idx_trojan=np.where(Trojaned==1)[0]
                idx_success=np.where(success==1)[0]
                trojan_success=[i for i in idx_success if i in idx_trojan]
                figname=task+'_'+advAttack#, figname=figname, label=backdoor_name
                print(figname)
                DR, FPR, FNR, AUC = outliers_detection(delta[idx_clean],delta[idx_trojan],t_tnr=t_tnr,figname=figname, label=backdoor_name)
                text+='&  %.2f/%.2f/%.2f'%((1-FNR)*100,FPR*100, (1-FNR)*(1-FPR)*2/ (2-FNR-FPR))
                torch.cuda.empty_cache()
            text+= '\n'
    print(text)
    
    
# Train_all_models()

# evaluate_all_models()
# ABC_evauation(0.85)
# STRIP_evaluation(0.85)
# Decision_Radius('flower',6,4)
#& 85.58 &  97.35 & 85.80 &  100.00 & 85.59 &  100.00
# cifar10  & 83.81 &  96.20 & 83.65 &  100.00
# for eps in [ 0.001]:
#     Train_all_models(robust=True, epsilon=eps,  TrainanyWay=True)
#     ABC_evauation(0.85, prefix='robust_eps_%.4f_'%eps)
 

In [1]:
import torch
torch.cuda.is_available()

/mnt/cluster/envs/templates/pytorch-1.11/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [16]:
for eps in [  0.1,0.01]:
    BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting('cifar10')
    tot_num = len(trainset)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=1)
    model=Model(gpu=True,num_class=class_num)
    model=model.cuda()
    train_model(model, train_loader, N_EPOCH, gpu=True,verbose=True, Robustness=True, epsilon=eps)
    ABC_evauation(0.85,prefix='robust_eps_%.4f_'%eps)
# BATCH_SIZE, N_EPOCH, trainset, testset,  Model, class_num = load_dataset_setting('gtsrb')
# train,test=[],[]
# for x,y in trainset:
#     train.append(x.cpu().numpy().transpose(1,2,0))
# for x,y in testset:
#     test.append(x.cpu().numpy().transpose(1,2,0))
# np.save('gtsrb.npy', {'train': train, 'test':test})

Files already downloaded and verified
| Wide-Resnet 22x10


KeyboardInterrupt: 

In [11]:
for eps in [ 0.01, 0.1]:
    evaluate_all_models(prefix='robust_eps_%.4f_'%eps)

Files already downloaded and verified
| Wide-Resnet 22x10
cifar10  & 79.23 &  96.03 
Files already downloaded and verified
| Wide-Resnet 22x10
cifar10  & 79.23 &  96.03 & 80.19 &  99.88 
Files already downloaded and verified
| Wide-Resnet 22x10
cifar10  & 79.23 &  96.03 & 80.19 &  99.88 & 80.00 &  100.00 
Files already downloaded and verified
| Wide-Resnet 22x10
cifar10  & 79.23 &  96.03 & 80.19 &  99.88 & 80.00 &  100.00 & 80.06 &  100.00 
Files already downloaded and verified
| Wide-Resnet 22x10
cifar10  & 79.23 &  96.03 & 80.19 &  99.88 & 80.00 &  100.00 & 80.06 &  100.00 & 79.81 &  100.00 
Files already downloaded and verified
| Wide-Resnet 22x10
cifar10  & 79.23 &  96.03 & 80.19 &  99.88 & 80.00 &  100.00 & 80.06 &  100.00 & 79.81 &  100.00 & 80.47 &  99.98 
cifar10  & 79.23 &  96.03 & 80.19 &  99.88 & 80.00 &  100.00 & 80.06 &  100.00 & 79.81 &  100.00 & 80.47 &  99.98 

Files already downloaded and verified
| Wide-Resnet 22x10
cifar10  & 36.35 &  53.74 
Files already downloaded 

In [35]:
print(torch.cuda.is_available())
!python trainmodels.py

True
| Wide-Resnet 22x10
| Wide-Resnet 22x10
../aten/src/ATen/native/cuda/Loss.cu:257: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [9,0,0] Assertion `t >= 0 && t < n_classes` failed.
Traceback (most recent call last):
  File "/mnt/cluster/data/users/arezoo/SP/trainmodels.py", line 199, in <module>
    Train_all_models()
  File "/mnt/cluster/data/users/arezoo/SP/trainmodels.py", line 181, in Train_all_models
    train_naive_Model(task,TrainanyWay=False)
  File "/mnt/cluster/data/users/arezoo/SP/trainmodels.py", line 117, in train_naive_Model
    train_model(model, train_loader, epoch_num=N_EPOCH,gpu=gpu, verbose=True)
  File "/mnt/cluster/data/users/arezoo/SP/trainmodels.py", line 88, in train_model
    cum_loss += loss.item() * B
RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
